In [3]:
import warnings
warnings.filterwarnings("ignore")


import glob
import logging
import os
import re
import uuid
from pathlib import Path
from pymongo import MongoClient
import artm
import click
import mlflow
from tqdm.notebook import tqdm
import nltk

import multiprocessing as mp

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
raw_data_path = os.path.join("..", "..", "data", "raw")
processed_data_path = os.path.join("..", "..", "data", "processed")
models_path = os.path.join("..", "..", "models")
experiments_path = os.path.join("..", "..", "experiments")

In [6]:
# Создание файла с данными для обучения

In [7]:
logging.info("Loading main dataset...")
client = MongoClient('localhost', 27017)
db = client.publicru_test
collection = db.documents_collection

2020-04-02 00:41:02,325 : INFO : Loading main dataset...


In [8]:
result = collection.find({}).skip(1000).limit(1)
document = result[0]

In [9]:
max_para = 5
max_sent = 10

text, bigrams, trigrams = [], [], []
for j, para in enumerate(document["t_body"][0]):
    for i, sent in enumerate(para):
        bigrams += list(nltk.bigrams(sent))
        trigrams += list(nltk.trigrams(sent))
        text += sent
        if i > max_sent:
            break
    if j > max_para:
        break
bigrams = ["!".join(b) for b in bigrams]
trigrams = ["!".join(t) for t in trigrams]
#             title = document['t_title'][0] if document['t_title'] else []
title = document['t_title'][0][0][0] if document['t_title'][0][0] else []
parts = [f"{document['_id']}"]
parts += ['|@title']  + title
parts += ['|@text']  + text
parts += ['|@bigrams']  + bigrams
parts += ['|@trigrams']  + trigrams
post = ' '.join(parts)

In [10]:
post

'5e7854ecc59124ce04bed321 |@title чат_S помощь_S |@text аукцион_S рубрика_S вести_V информационный_A центр_S ъ рисунок_S сергей_S голосов_S покупатель_S любить_V общаться_V интернет_S магазин_S данные_S последний_A опрос_S проводить_V facebook половина_S потребитель_S предпочитать_V онлайн_ADV покупка_S сайт_S функция_S чат_S вывод_S коммуникация_S мессенджер_S набирать_V сила_S собираться_V замедляться_V будущее_S респондент_S пользоваться_V мессенджер_S коммерческий_A цель_S посылать_V сообщение_S ожидать_V активный_A переписка_S следующий_A использование_S приложение_S переписка_S являться_V прерогатива_S молодой_A миллениал_S предпочитать_V мессенджер_S звонок_S почта_S отставать_V родитель_S поколение_S х_S практически_ADV вровень_ADV идти_V бебибумер_S рождаться_V мировой_A война_S покупатель_S любить_V пользоваться_V услуга_S обратный_A связь_S быстрый_A отклик_S спрашивать_V товар_S услуга_S местонахождение_S часы_S работа_S магазин_S взаимодействие_S онлайн_ADV магазин_S мессе

In [11]:
def create_corpus_file(collection, path_folder):
    with open(os.path.join(experiments_path, path_folder, 'vowpal_wabbit_corpus.txt'), 'w') as the_file:
        mongo_filter = {
            "t_title": {"$exists": True}, 
            "t_body": {"$exists": True}, 
            "words_count": {"$gt": 400}
        }
        max_para = 5
        max_sent = 10
        for document in tqdm(
                collection.find(mongo_filter, {"_id": 1, "t_title": 1, "t_body": 1}, no_cursor_timeout=True)):
            text, bigrams, trigrams = [], [], []
            for j, para in enumerate(document["t_body"][0]):
                for i, sent in enumerate(para):
                    bigrams += list(nltk.bigrams(sent))
                    trigrams += list(nltk.trigrams(sent))
                    text += sent
                    if i > max_sent:
                        break
                if j > max_para:
                    break
            bigrams = ["!".join(b) for b in bigrams]
            trigrams = ["!".join(t) for t in trigrams]
#             title = document['t_title'][0] if document['t_title'] else []
            title = document['t_title'][0]
            if title:
                title = title[0][0] if title[0] else []
            parts = [f"{document['_id']}"]
            parts += ['|@title']  + title
            parts += ['|@text']  + text
            parts += ['|@bigrams']  + bigrams
            parts += ['|@trigrams']  + trigrams
            post = ' '.join(parts)
            the_file.write(f"{post}\n")

In [12]:
def preparing_batch(path_folder):
    if len(glob.glob(os.path.join(experiments_path, path_folder, 'batches', '*.batch'))) > 1:
        logging.info("Remove old batches.")
        pth = Path(os.path.join(experiments_path, path_folder, 'batches'))
        for child in pth.glob('*'):
            if child.is_file():
                child.unlink()
        pth.rmdir()

    logging.info("Generate batches files...")
    batch_vectorizer = artm.BatchVectorizer(
        data_path=os.path.join(experiments_path, path_folder, 'vowpal_wabbit_corpus.txt'),
        data_format='vowpal_wabbit',
        batch_size=5000,
        target_folder=os.path.join(experiments_path, path_folder, 'batches'),
    )
    logging.info("Gathering dictionary...")
    dictionary = batch_vectorizer.dictionary
    dictionary.save_text(dictionary_path=os.path.join(experiments_path, path_folder, 'dictionary.txt'))

In [13]:
data_folder = "multiclass_model_test_all"

In [14]:
# mongo_filter = {"words_count": {"$gt": 400}}
mongo_filter = {"words_count": {"$gt": 400}}
logging.info(f"Original dataset size: {collection.count_documents({})}")
logging.info(f"Working dataset size: {collection.count_documents(mongo_filter)}")

2020-04-02 00:45:41,451 : INFO : Original dataset size: 588456
2020-04-02 00:45:42,324 : INFO : Working dataset size: 226590


In [15]:
logging.info("Creating corpus file in vw format...")
Path(os.path.join(experiments_path, data_folder)).mkdir(parents=True, exist_ok=True)
create_corpus_file(collection, data_folder)
logging.info("Done.")

2020-04-02 00:45:45,338 : INFO : Creating corpus file in vw format...


2020-04-02 00:48:36,248 : INFO : Done.


In [16]:
logging.info("Preparing batches for artm model...")
preparing_batch(data_folder)
logging.info("Finish.")

2020-04-02 00:48:36,258 : INFO : Preparing batches for artm model...
2020-04-02 00:48:36,261 : INFO : Remove old batches.
2020-04-02 00:48:37,693 : INFO : Generate batches files...
2020-04-02 01:01:21,712 : INFO : Gathering dictionary...


InvalidOperationException: Unable to serialize the message

In [ ]:
!ls -lah ../../experiments/multiclass_model_test_all/

In [ ]:
def load_batches(path_folder, min_df):
    """
    path_folder: string, folder with vowpal_wabbit_corpus.txt and batches/
    min_df: integer, minimal document frequency 
    """
    if len(glob.glob(os.path.join(experiments_path, path_folder, 'batches', '*.batch'))) < 1:
        logging.info("Generating batches files...")
        bv = artm.BatchVectorizer(
            data_path=os.path.join(experiments_path, path_folder, 'vowpal_wabbit_corpus.txt'),
            data_format='vowpal_wabbit',
            batch_size=2000,
            target_folder=os.path.join(experiments_path, path_folder, 'batches'),
        )
        dictionary = bv.dictionary
        dictionary.save_text(dictionary_path=os.path.join(experiments_path, path_folder, 'dictionary.txt'))
    else:
        logging.info("Loading batches files...")
        bv = artm.BatchVectorizer(
            data_path=os.path.join(experiments_path, path_folder, 'batches'),
            data_format='batches',
        )
        dictionary = artm.Dictionary()
        dictionary.load_text(dictionary_path=os.path.join(experiments_path, path_folder, 'dictionary.txt'))

    regex = "\d+"
    match = re.findall(regex, str(dictionary))
    logging.info(f"Original dictionary size: {int(match[-1])} words.")
    dictionary.filter(min_df=min_df, inplace=True)
    match = re.findall(regex, str(dictionary))
    logging.info(f"Filtered dictionary size: {int(match[-1])} words.")
    return bv, dictionary


In [ ]:
def create_topic_names(topic_count=200, background_topic_count=20):
    objective_topics = ['objective_topic_' + str(x) for x in range(0, topic_count - background_topic_count)]
    background_topics = ['background_topic_' + str(x) for x in range(topic_count - background_topic_count, topic_count)]
    all_topics = objective_topics + background_topics

    return all_topics, objective_topics, background_topics

In [ ]:
def print_measures(model):
    logging.info('Sparsity Title Phi: {0:.3f}'.format(model.score_tracker['SparsityPhiTitleScore'].last_value))
    logging.info('Sparsity Text Phi: {0:.3f}'.format(model.score_tracker['SparsityPhiTextScore'].last_value))
    logging.info('Sparsity Bigrams Phi: {0:.3f}'.format(model.score_tracker['SparsityPhiBigramsScore'].last_value))
    logging.info('Sparsity Trigrams Phi: {0:.3f}'.format(model.score_tracker['SparsityPhiTrigramsScore'].last_value))
    logging.info('Sparsity Theta: {0:.3f}'.format(model.score_tracker['SparsityThetaScore'].last_value))
    logging.info('Kernel title contrast: {0:.3f}'.format(model.score_tracker['TopicKernelTitleScore'].last_average_contrast))
    logging.info('Kernel text contrast: {0:.3f}'.format(model.score_tracker['TopicKernelTextScore'].last_average_contrast))
    logging.info('Kernel bigrams contrast: {0:.3f}'.format(model.score_tracker['TopicKernelBigramsScore'].last_average_contrast))
    logging.info('Kernel trigrams contrast: {0:.3f}'.format(model.score_tracker['TopicKernelTrigramsScore'].last_average_contrast))
    logging.info('Kernel title purity: {0:.3f}'.format(model.score_tracker['TopicKernelTitleScore'].last_average_purity))
    logging.info('Kernel text purity: {0:.3f}'.format(model.score_tracker['TopicKernelTextScore'].last_average_purity))
    logging.info('Kernel bigrams purity: {0:.3f}'.format(model.score_tracker['TopicKernelBigramsScore'].last_average_purity))
    logging.info('Kernel trigrams purity: {0:.3f}'.format(model.score_tracker['TopicKernelTrigramsScore'].last_average_purity))
    logging.info('Perplexity: {0:.3f}'.format(model.score_tracker['PerplexityScore'].last_value))

In [ ]:
def generate_name():
    return str(uuid.uuid1())

In [ ]:
def mlflow_log_metrics(model):
    mlflow.log_metrics({
        "DeccorPhi": model.regularizers['DeccorPhi'].tau,
        "SmoothPhi": model.regularizers['SmoothPhi'].tau,
        "SmoothTheta": model.regularizers['SmoothTheta'].tau,
        "SparsePhi": model.regularizers['SparsePhi'].tau,
        "SparseTheta": model.regularizers['SparseTheta'].tau,
        "SparsityPhiTitleScore": model.score_tracker['SparsityPhiTitleScore'].last_value,
        "SparsityPhiTextScore": model.score_tracker['SparsityPhiTextScore'].last_value,
        "SparsityPhiBigramsScore": model.score_tracker['SparsityPhiBigramsScore'].last_value,
        "SparsityPhiTrigramsScore": model.score_tracker['SparsityPhiTrigramsScore'].last_value,
        "SparsityThetaScore": model.score_tracker['SparsityThetaScore'].last_value,
        "KernelContrastTitleScore": model.score_tracker['TopicKernelTitleScore'].last_average_contrast,
        "KernelContrastTextScore": model.score_tracker['TopicKernelTextScore'].last_average_contrast,
        "KernelContrastBigramsScore": model.score_tracker['TopicKernelBigramsScore'].last_average_contrast,
        "KernelContrastTrigramsScore": model.score_tracker['TopicKernelTrigramsScore'].last_average_contrast,
        "TopicPurityTitleScore": model.score_tracker['TopicKernelTitleScore'].last_average_purity,
        "TopicPurityTextScore": model.score_tracker['TopicKernelTextScore'].last_average_purity,
        "TopicPurityBigramsScore": model.score_tracker['TopicKernelBigramsScore'].last_average_purity,
        "TopicPurityTrigramsScore": model.score_tracker['TopicKernelTrigramsScore'].last_average_purity,
        "PerplexityScore": model.score_tracker['PerplexityScore'].last_value,
    }, step=model.num_phi_updates)

In [ ]:
def next_step(i, model, batch_vectorizer, step_size, dataset_name):
    model_name = generate_name()
    logging.info(model_name)

    for _ in tqdm(range(step_size)):
        model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=1)
        mlflow_log_metrics(model)
    print_measures(model)

    Path(os.path.join(experiments_path, 'models')).mkdir(parents=True, exist_ok=True)
    model_dir_name = os.path.join(experiments_path, 'models', f"{model_name}")
    model.dump_artm_model(model_dir_name)
    mlflow.set_tag(f"model_dump_{i}", model_dir_name)
    mlflow.log_artifacts(model_dir_name)

In [ ]:
dataset_name = "multiclass_model_test_all"
min_df = 6
num_all_topics = 400
num_background_topics = 20
step_size = 10

In [ ]:
bv, dictionary = load_batches(dataset_name, min_df)

In [ ]:
all_topics, objective_topics, background_topics = create_topic_names(num_all_topics, num_background_topics)

In [ ]:
scores_artm = [
        artm.PerplexityScore(name='PerplexityScore', dictionary=dictionary, class_ids=["@title", "@text", "@bigrams", "@trigrams"]),
        artm.SparsityPhiScore(name='SparsityPhiTitleScore', topic_names=objective_topics, class_id="@title"),
        artm.SparsityPhiScore(name='SparsityPhiTextScore', topic_names=objective_topics, class_id="@text"),
        artm.SparsityPhiScore(name='SparsityPhiBigramsScore', topic_names=objective_topics, class_id="@bigrams"),
        artm.SparsityPhiScore(name='SparsityPhiTrigramsScore', topic_names=objective_topics, class_id="@trigrams"),
        artm.SparsityThetaScore(name='SparsityThetaScore', topic_names=objective_topics),
        artm.TopTokensScore(name='TopTokensTitleScore', num_tokens=20, topic_names=objective_topics, dictionary=dictionary, class_id="@title"),
        artm.TopTokensScore(name='TopTokensTextScore', num_tokens=20, topic_names=objective_topics, dictionary=dictionary, class_id="@text"),
        artm.TopTokensScore(name='TopTokensBigramsScore', num_tokens=20, topic_names=objective_topics, dictionary=dictionary, class_id="@bigrams"),
        artm.TopTokensScore(name='TopTokensTrigramsScore', num_tokens=20, topic_names=objective_topics, dictionary=dictionary, class_id="@trigrams"),
        artm.TopicKernelScore(name='TopicKernelTitleScore', class_id="@title", probability_mass_threshold=0.25,
                              topic_names=objective_topics, dictionary=dictionary),
        artm.TopicKernelScore(name='TopicKernelTextScore', class_id="@text", probability_mass_threshold=0.25,
                              topic_names=objective_topics, dictionary=dictionary),
        artm.TopicKernelScore(name='TopicKernelBigramsScore', class_id="@bigrams", probability_mass_threshold=0.25,
                              topic_names=objective_topics, dictionary=dictionary),
        artm.TopicKernelScore(name='TopicKernelTrigramsScore', class_id="@trigrams", probability_mass_threshold=0.25,
                              topic_names=objective_topics, dictionary=dictionary),
    ]

In [ ]:
regularizers_artm = [
        artm.DecorrelatorPhiRegularizer(name='DeccorPhi', topic_names=objective_topics, gamma=0, tau=0),
        artm.SmoothSparsePhiRegularizer(name='SparsePhi', topic_names=objective_topics, dictionary=dictionary, gamma=0,
                                        tau=0),
        artm.SmoothSparsePhiRegularizer(name='SmoothPhi', topic_names=background_topics, dictionary=dictionary, gamma=0,
                                        tau=0),
        artm.SmoothSparseThetaRegularizer(name='SparseTheta', topic_names=objective_topics, tau=0),
        artm.SmoothSparseThetaRegularizer(name='SmoothTheta', topic_names=background_topics, tau=0),
        artm.TopicSelectionThetaRegularizer(name='TopicSelectionTheta', topic_names=objective_topics, tau=0)
    ]

In [ ]:
model = artm.ARTM(
        num_topics=num_all_topics,
        topic_names=all_topics,
        class_ids={'@title': 3.0, '@text': 1.0, "@bigrams": 2.0, "@trigrams": 4.0},
        num_processors=mp.cpu_count() - 1,
        num_document_passes=2,
        regularizers=regularizers_artm,
        scores=scores_artm,
        dictionary=dictionary,
        cache_theta=False,
        seed=42,
        show_progress_bars=False
    )

In [ ]:
logging.info("Run learning...")
mlflow.set_experiment(dataset_name)
with mlflow.start_run():

    # этап 1 - сильная декорреляция + сглаживание
    # Sparse < 0
    # Smooth > 0
    model.regularizers['DeccorPhi'].tau = 0.005
    model.regularizers['SmoothPhi'].tau = 0.4
    model.regularizers['SmoothTheta'].tau = 0.4
    next_step(1, model, bv, step_size, dataset_name)

    model.regularizers['DeccorPhi'].tau = 0.015
    model.regularizers['SmoothPhi'].tau = 0.6
    model.regularizers['SmoothTheta'].tau = 0.6
    next_step(2, model, bv, step_size, dataset_name)

    model.regularizers['DeccorPhi'].tau = 0.03
    model.regularizers['SmoothPhi'].tau = 0.8
    model.regularizers['SmoothTheta'].tau = 0.8
    next_step(3, model, bv, step_size, dataset_name)

    # этап 2 - подключение разреживания предметных, постепенное увеличение разреживания
    # Sparse < 0
    # Smooth > 0
    model.regularizers['SparsePhi'].tau = -0.0001
    model.regularizers['SparseTheta'].tau = -0.1
    next_step(4, model, bv, step_size, dataset_name)

    model.regularizers['SparsePhi'].tau = -0.0002
    model.regularizers['SparseTheta'].tau = -0.2
    next_step(5, model, bv, step_size, dataset_name)

    model.regularizers['SparsePhi'].tau = -0.0003
    model.regularizers['SparseTheta'].tau = -0.3
    next_step(6, model, bv, step_size, dataset_name)

    # этап 3
    # Sparse < 0
    # Smooth > 0
    model.regularizers['SparsePhi'].tau = -0.0005
    model.regularizers['SparseTheta'].tau = -0.4
    next_step(7, model, bv, step_size, dataset_name)

    model.regularizers['SparsePhi'].tau = -0.0006
    model.regularizers['SparseTheta'].tau = -0.5
    next_step(8, model, bv, step_size, dataset_name)

    model.regularizers['SparsePhi'].tau = -0.0007
    model.regularizers['SparseTheta'].tau = -0.6
    next_step(9, model, bv, step_size, dataset_name)

logging.info("Finish")

In [ ]:
%%time

with open("topics_print.txt","w") as f:
    topics_tokens = ""
    for i, topic_name in enumerate(model.topic_names[:190]):
        f.write(f"topic_name: {i+1}\n\n")

        titles = " ".join([word.split("_")[0] for word in model.score_tracker['TopTokensTitleScore'].last_tokens[topic_name]])
        f.write(f"title keywords: {titles}\n\n")
       
        texts = " ".join([word.split("_")[0] for word in model.score_tracker['TopTokensTextScore'].last_tokens[topic_name]])
        f.write(f"text keywords: {texts}\n\n")

        bigrams = []
        for bigram in model.score_tracker['TopTokensBigramsScore'].last_tokens[topic_name]:
            words = []
            for word in bigram.split("!"):
                words.append(word.split("_")[0])
            bigrams.append("_".join(words))
        f.write(f"bigram keywords: {' '.join(bigrams)}\n\n")
        
        trigrams = []
        for trigram in model.score_tracker['TopTokensTrigramsScore'].last_tokens[topic_name]:
            words = []
            for word in trigram.split("!"):
                words.append(word.split("_")[0])
            trigrams.append("_".join(words))
        f.write(f"trigram keywords: {' '.join(trigrams)}\n{'-'*100}\n\n\n\n")